# Issue with contiguity masking Sentinel-2 at 10m resolution

## Sample query

In [ ]:
import datacube
dc = datacube.Datacube(app='S2-Landsat-Comparison')

In [ ]:
# This works correctly
ls7_wo_noinit = dc.load(product='ga_ls_wo_3', id='5ebee006-33d8-5841-8c5f-23721b0506a5', output_crs='epsg:32752', resolution=(-10, 10), resampling='nearest')

In [ ]:
ls7_wo_init = dc.load(product='ga_ls_wo_3', id='5ebee006-33d8-5841-8c5f-23721b0506a5', output_crs='+init=epsg:32752', resolution=(-10, 10), resampling='nearest')

In [ ]:
# rasterio.warp.calculate_default_transform('EPSG:32752', 'EPSG:3577', 10980, 10980, 699960.0, 6790240.0, 809760.0, 6900040.0)
ls7_wo = dc.load(product='ga_ls_wo_3', id='5ebee006-33d8-5841-8c5f-23721b0506a5', output_crs='+init=epsg:32752', resolution=(-10, 10), resampling='nearest')

In [ ]:
ls7_wo = dc.load(product='ga_ls_wo_3', id='5ebee006-33d8-5841-8c5f-23721b0506a5', output_crs='epsg:32752', resolution=(-10, 10), resampling='nearest')

In [ ]:
import odc.algo
import datacube 
import xarray as xr
import matplotlib.pyplot as plt

import sys
sys.path.append('../Scripts')
from dea_dask import create_local_dask_cluster

dc = datacube.Datacube()



# Jervis Bay
ymax, xmin = -35.075, 150.687
ymin, xmax = -35.135, 150.712

# Jervis Bay
ymax, xmin = -35.00, 150.60
ymin, xmax = -35.20, 150.80

# # # Gold Coast
# ymax, xmin = -28.086443, 153.445244
# ymin, xmax = -28.274374, 153.598194


# # Sydney
# ymax, xmin = -33.825292, 151.192090
# ymin, xmax = -34.014403, 151.309498

# # Perth
# ymax, xmin = -31.658507, 115.657332
# ymin, xmax = -31.951340, 115.955074

query = {
    'x': (xmin, xmax),
    'y': (ymax, ymin),
    'time': ('2018-01', '2018-01'),
    'measurements': ['nbart_red', 'nbart_green', 'nbart_blue', 'fmask', 'nbart_contiguity'],
    'output_crs': 'EPSG:32756',
    'group_by': 'solar_day',
    'resolution': (-30, 30),
}


create_local_dask_cluster()


In [ ]:
import odc.algo
import datacube 
import xarray as xr
import matplotlib.pyplot as plt

import sys
sys.path.append('../Scripts')
from dea_dask import create_local_dask_cluster

dc = datacube.Datacube()


# Flowchart example
ymax, xmin = -35.84, 150.135
ymin, xmax = -35.92, 150.205


query = {
    'x': (xmin, xmax),
    'y': (ymax, ymin),
    'time': ('2018-01', '2018-12'),
    'measurements': ['nbart_swir_1', 'nbart_green'],
    'output_crs': 'EPSG:32755',
    'group_by': 'solar_day',
#     'resampling': 'average',
    'resolution': (-30, 30),
}


ds = dc.load(product=['ga_ls8c_ard_3'],
             dask_chunks={'x': 3000, 'y': 3000, 'time': 1},
             **query)
ds

In [ ]:
mndwi = (ds.nbart_green - ds.nbart_swir_1) / (ds.nbart_green + ds.nbart_swir_1)
# mndwi.plot(robust=True, col='time', col_wrap=3)

In [ ]:
mndwi.isel(time=19).plot(robust=True, aspect=0.9, size=15, cmap='RdBu')
plt.gcf().savefig('out.png', dpi=95)

In [ ]:
import datacube 
# from skimage.morphology import binary_opening, disk
import xarray as xr
import numpy as np

ds = dc.load(product=['ga_ls7e_ard_3'],  #['ga_ls7e_ard_3'],
             dask_chunks={'x': 3000, 'y': 750, 'time': 1},
             **query)
ds


In [ ]:
import odc.algo

In [ ]:
import dask
dask.__version__

In [ ]:
pip install --upgrade dask

In [ ]:
known_cloud =  (ds.fmask == 2) | (ds.fmask == 3) | (ds.fmask == 4)
odc.algo.binary_closing(known_cloud, 10)

In [ ]:
# ds.fmask.plot(col='time')

In [ ]:
known_cloud.plot(col='time', size=7)

In [ ]:
known_cloud =  (ds.fmask == 2) | (ds.fmask == 3) | (ds.fmask == 4)
known_clear = ds.fmask != 0
known_cloud.plot(col='time', size=5)

In [ ]:
cloud_dilated = odc.algo.binary_closing(known_cloud, 10)
cloud_gapfilled = known_cloud.where(known_clear, cloud_dilated)
cloud_gapfilled.plot(col='time', size=5)

In [ ]:
odc.algo.binary_opening(cloud_gapfilled, 10).plot(col='time', size=5)

In [ ]:
mask.where(~mask_nodata, odc.algo.binary_closing(mask, 10)).plot(col='time', size=7)

In [ ]:
mask =  (ds.fmask == 0) | (ds.fmask == 2) | (ds.fmask == 3) | (ds.fmask == 4)
mask.mean(dim='time').plot(size=10, vmin=0, vmax=1.0)

In [ ]:
import odc.algo

mask = odc.algo.enum_to_bool(ds.fmask, categories=['nodata', 'cloud', 'shadow', 'snow'])
mask = odc.algo.binary_closing(mask, 2)
mask_cleaned = odc.algo.mask_cleanup(mask, r=(10, 5))
ds_masked = odc.algo.erase_bad(ds, mask_cleaned)

In [ ]:
(ds_masked.nbart_red == -999).plot(col='time')  #.mean(dim='time').plot(size=10, vmin=0, vmax=1.0)

In [ ]:
erase_bad(ds, ((ds.fmask == 0) | (ds.fmask == 2) | (ds.fmask == 3) | (ds.fmask == 4))).nbart_red.plot(col='time', size=10)

In [ ]:
from odc.algo import erase_bad

In [ ]:
.isel(time=0).nbart_red.plot()

In [ ]:

mask_closed.plot(col='time')

In [ ]:
  #

In [ ]:
mask_closed.plot(col='time')

In [ ]:
mask_cleaned.where(ds.fmask != 0, 1).plot(col='time')

In [ ]:
(ds.fmask == 0).plot(col='time')

In [ ]:
from odc.algo import binary_dilation

In [ ]:
from skimage.morphology import binary_opening, disk
import xarray as xr
import numpy as np

mask = (ds.fmask == 0).load()

kernel = disk(5)
mask = xr.apply_ufunc(binary_opening,
                      mask,
                      kernel.reshape((1,) + kernel.shape),
                      output_dtypes=[np.bool],
                      dask='parallelized',
                      keep_attrs=True)

## Load and preview data at 20m resolution

In [ ]:
# Load data
ds = dc.load(product='s2b_ard_granule', 
#              resolution=(-20, 20),
             **query)

## Create basic fmask mask

In [ ]:
# ds.fmask.plot(col='time', col_wrap=4)

In [ ]:
mask = odc.algo.fmask_to_bool(ds.fmask, categories=['valid', 'snow', 'water'])

In [ ]:
# mask.plot(col='time', col_wrap=4)

## Create cleaned and buffered mask

In [ ]:
from skimage.morphology import binary_erosion, binary_dilation, binary_closing, disk

# def cloud_buffer(da, radius=0, base_radius=10):

#     # First remove small or narrow features using morphological closing
#     mask_closing = binary_closing(da, disk(base_radius))
    
#     # Then buffer remaining pixels by a radius to mask cloud edges
#     mask_buffer = binary_erosion(mask_closing, disk(radius))
    
#     return mask_buffer

def cloud_buffer(da, dilate_radius=15, erode_radius=5):

    # First remove small or narrow features using morphological closing
    mask_dilated = binary_dilation(da, disk(dilate_radius))
    
    # Then buffer remaining pixels by a radius to mask cloud edges
    mask_eroded = binary_erosion(mask_dilated, disk(erode_radius))
    
    return mask_eroded



In [ ]:
new_mask = mask.groupby('time').apply(cloud_buffer)
cloudy_scenes = ds.where(new_mask).nbart_blue.mean(dim=['x', 'y']) < 3000

# Compute median RGB and sum of good data through time
rgb_mask = ds.where(mask).sel(time=cloudy_scenes)[['nbart_red', 'nbart_green', 'nbart_blue']].median(dim='time')
rgb_new_mask = ds.where(new_mask).sel(time=cloudy_scenes)[['nbart_red', 'nbart_green', 'nbart_blue']].median(dim='time')
sum_mask = mask.sel(time=cloudy_scenes).sum(dim='time')
sum_new_mask = new_mask.sel(time=cloudy_scenes).sum(dim='time')


# Create four panel figure
fig, axes = plt.subplots(2, 2, figsize=(12, 15))

# Plot median RGB composites into left column
rgb_mask.to_array().plot.imshow(robust=True, ax=axes[0, 0])
rgb_new_mask.to_array().plot.imshow(robust=True, ax=axes[1, 0])

# Calculate count stats and plot count arrays into right column
vmax = xr.concat([sum_mask.max(), sum_new_mask.max()], dim='time').max().item()
sum_mask.plot(vmin=0, vmax=vmax, ax=axes[0, 1], cmap='YlOrRd_r')
sum_new_mask.plot(vmin=0, vmax=vmax, ax=axes[1, 1], cmap='YlOrRd_r')

# Set titles
axes[0, 0].set_title("RGB; fmask")
axes[0, 1].set_title("Good data count; fmask")
axes[1, 0].set_title("RGB; fmask with filtering")
axes[1, 1].set_title("Good data count; fmask with filtering");

In [ ]:
from datacube.utils.cog import write_cog
write_cog(rgb_mask.to_array(), fname='test_old_mask_syd.tif')
write_cog(rgb_new_mask.to_array(), fname='test_new_mask_syd.tif')

In [ ]:
# raster = xr.open_rasterio('test2.tif')
# write_cog(raster.isel(band=0), fname='raster.tif')


In [ ]:
import xarray as xr
import datacube

# Connect to datacube
dc = datacube.Datacube()

# Load raster
raster = xr.open_rasterio("raster.tif").squeeze("band")

# Load data from datacube. `like=raster.geobox` step will load data
# from the datacube using the same resolution and CRS of `raster`
ds = dc.load(
    product="s2b_ard_granule",
    measurements=["nbart_red"],
    time=("2018-01", "2018-02"),
    like=raster.geobox,
    resampling="nearest",
    group_by="solar_day",
)

# Use raster data as a mask
ds.where(raster > 500)

In [ ]:
ds